In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import upload_data, pull_model, pull_data

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get model from our Hopsworks model registry

In [3]:
MODEL_NAME = 'random_forest_ohlc' # this one predicts tmw_1_0_percent_increase_binary

# ***** later implement code that checks the latest model version and pulls that one
# def pull_model(model_name, model_version):
model = pull_model(MODEL_NAME, 1)

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


### Get the full Ethereum data from Hopsworks feature store.

In [5]:
eth_ohlc = pull_data('eth_ohlc_transformed', 1, 'eth_ohlc_transformed_view', 1)

eth_ohlc

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.80s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
806,2015-11-15 00:00:00+00:00,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,0.936850,0.032503,...,0,1,0,0,0,0,0,0,0,1
1225,2015-11-16 00:00:00+00:00,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.020000,0.089216,...,0,1,0,1,0,0,0,0,0,0
2432,2015-11-17 00:00:00+00:00,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,1.000000,-0.010000,...,0,1,0,0,1,0,0,0,0,0
1968,2015-11-18 00:00:00+00:00,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.982750,-0.007377,...,0,1,0,0,0,1,0,0,0,0
859,2015-11-19 00:00:00+00:00,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.940300,-0.016165,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751,2024-04-18 00:00:00+00:00,2986.000000,3087.000000,2960.000000,3064.000000,1.350000e+10,4.453302e+06,3.706000e+11,3101.000000,0.011932,...,0,0,0,0,0,0,1,0,0,0
2793,2024-04-19 00:00:00+00:00,3061.000000,3117.000000,2879.000000,3085.000000,1.740000e+10,5.675824e+06,3.746000e+11,3160.500000,0.023889,...,0,0,0,0,0,0,0,1,0,0
2705,2024-04-20 00:00:00+00:00,3085.000000,3166.000000,3025.000000,3155.000000,8.900000e+09,2.884086e+06,3.776000e+11,3168.500000,0.004261,...,0,0,0,0,0,0,0,0,1,0
951,2024-04-21 00:00:00+00:00,3153.000000,3191.000000,3120.000000,3146.000000,8.200000e+09,2.603957e+06,3.867000e+11,3215.178412,0.021516,...,0,0,0,0,0,0,0,0,0,1


### Make Predictions for all the days and add this predictions column to the dataframe

In [6]:
predictors = []
periods = [2, 5, 10, 25, 50, 100]
percent_increase_counts = ['1_0', '1_25', '1_5', '1_75', '2_0', '2_25', '2_5', '2_75', '3_0']
for period in periods:
    for indicator in ['ema', 'rsi', 'sma']:
        predictors.append(f'{indicator}_{period}')
    for percent in percent_increase_counts:
        predictors.append(f'last_{period}_day_{percent}_percent_increase_count')
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
predictors.extend(months)
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
predictors.extend(weekdays)

x0 = eth_ohlc[predictors]
x0

,ema_2,rsi_2,sma_2,last_2_day_1_0_percent_increase_count,last_2_day_1_25_percent_increase_count,last_2_day_1_5_percent_increase_count,last_2_day_1_75_percent_increase_count,last_2_day_2_0_percent_increase_count,last_2_day_2_25_percent_increase_count,last_2_day_2_5_percent_increase_count,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
806,1.005643,63.673722,1.009804,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,1,0,0,0,0,0,0,0,1
1225,1.010034,80.442256,1.012313,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0,1,0,1,0,0,0,0,0,0
2432,1.030692,95.461059,1.041774,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0,1,0,0,1,0,0,0,0,0
1968,1.003404,69.213788,0.990000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,1,0,0,0,1,0,0,0,0
859,0.989253,35.519995,0.982267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751,1.004777,55.265104,1.013228,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,1,0,0,0
2793,1.003858,64.503124,1.003415,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,1,0,0
2705,1.008724,85.064674,1.011218,2.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
951,1.001941,74.036888,0.998572,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
predictions = model.predict(x0)

eth_ohlc['pred_tmw_1_0_percent_increase_binary'] = predictions
eth_ohlc['pred_tmw_1_0_percent_increase_binary'] = eth_ohlc['pred_tmw_1_0_percent_increase_binary'].astype('Int64')

### Select just the rows that will be used in the Streamlit Dashboard

In [18]:
print(eth_ohlc.columns.tolist())

['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap', 'tmw_avg_high_close', 'tmw_percent_increase', 'tmw_1_0_percent_increase_binary', 'tmw_1_25_percent_increase_binary', 'tmw_1_5_percent_increase_binary', 'tmw_1_75_percent_increase_binary', 'tmw_2_0_percent_increase_binary', 'tmw_2_25_percent_increase_binary', 'tmw_2_5_percent_increase_binary', 'tmw_2_75_percent_increase_binary', 'tmw_3_0_percent_increase_binary', 'tmw_positive_percent_increase_binary', 'ema_2', 'rsi_2', 'sma_2', 'last_2_day_1_0_percent_increase_count', 'last_2_day_1_25_percent_increase_count', 'last_2_day_1_5_percent_increase_count', 'last_2_day_1_75_percent_increase_count', 'last_2_day_2_0_percent_increase_count', 'last_2_day_2_25_percent_increase_count', 'last_2_day_2_5_percent_increase_count', 'last_2_day_2_75_percent_increase_count', 'last_2_day_3_0_percent_increase_count', 'ema_5', 'rsi_5', 'sma_5', 'last_5_day_1_0_percent_increase_count', 'last_5_day_1_25_percent_increase_count', 'last_

In [20]:
vars = ['date', 'open', 'low', 'high', 'close', 'tmw_avg_high_close', 'tmw_percent_increase',
        'pred_tmw_1_0_percent_increase_binary', 'tmw_1_0_percent_increase_binary']

eth_final = eth_ohlc[vars]

eth_final

,date,open,low,high,close,tmw_avg_high_close,tmw_percent_increase,pred_tmw_1_0_percent_increase_binary,tmw_1_0_percent_increase_binary,tmw_1_25_percent_increase_binary,tmw_1_5_percent_increase_binary,tmw_1_75_percent_increase_binary,tmw_2_0_percent_increase_binary,tmw_2_25_percent_increase_binary,tmw_2_5_percent_increase_binary,tmw_2_75_percent_increase_binary,tmw_3_0_percent_increase_binary,tmw_positive_percent_increase_binary
806,2015-11-15 00:00:00+00:00,0.891200,0.875000,0.921500,0.906400,0.936850,0.032503,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1225,2015-11-16 00:00:00+00:00,0.906200,0.892000,0.944700,0.929000,1.020000,0.089216,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2432,2015-11-17 00:00:00+00:00,0.924900,0.905800,1.030000,1.010000,1.000000,-0.010000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1968,2015-11-18 00:00:00+00:00,0.990000,0.940500,1.010000,0.990000,0.982750,-0.007377,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
859,2015-11-19 00:00:00+00:00,0.988700,0.937500,1.010000,0.955500,0.940300,-0.016165,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751,2024-04-18 00:00:00+00:00,2986.000000,2960.000000,3087.000000,3064.000000,3101.000000,0.011932,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2793,2024-04-19 00:00:00+00:00,3061.000000,2879.000000,3117.000000,3085.000000,3160.500000,0.023889,0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
2705,2024-04-20 00:00:00+00:00,3085.000000,3025.000000,3166.000000,3155.000000,3168.500000,0.004261,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
951,2024-04-21 00:00:00+00:00,3153.000000,3120.000000,3191.000000,3146.000000,3215.178412,0.021516,0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


### Upload to Hopsworks

In [22]:
# make sure some of the column types are what hopsworks wants
eth_final['date'] = pd.to_datetime(eth_final['date']).dt.tz_localize(None)
columns = [
    'tmw_1_0_percent_increase_binary',
    'tmw_1_25_percent_increase_binary',
    'tmw_1_5_percent_increase_binary',
    'tmw_1_75_percent_increase_binary',
    'tmw_2_0_percent_increase_binary',
    'tmw_2_25_percent_increase_binary',
    'tmw_2_5_percent_increase_binary',
    'tmw_2_75_percent_increase_binary',
    'tmw_3_0_percent_increase_binary',
    'tmw_positive_percent_increase_binary'
]
eth_final[columns] = eth_final[columns].astype('Int64')

upload_data(eth_final, 'eth_ohlc_predictions', 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/582805/fs/578628/fg/743893


Uploading Dataframe: 100.00% |██████████| Rows 3063/3063 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: eth_ohlc_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/eth_ohlc_predictions_1_offline_fg_materialization/executions
